# Notebook: E-handelsplattformen (Grupp 1: Daniel, Hami, Martin W)

Vi är ett team som analyserar försäljningsdata från en påhittad e-handelsplattform.  
Vi ska ge ledningen ett snabbt beslutsunderlag inför nästa kampanjperiod.

Datasetet innehåller:
- `order_id`
- `date`
- `city`
- `category`
- `price`
- `units`
- `revenue`

In [ ]:
# För att få relative path utifrån notebookens workspace mapp.
import sys, os 
sys.path.append(os.path.abspath(".."))

In [ ]:
# Läs in data
from src.io_utils import load_csv
df = load_csv("../data/ecommerce_sales.csv")
# Kolla 2 rader
df.head(2)

---

Vi ska svara följande affärsfrågor:
1. Vad säljer? – Vilka kategorier driver mest intäkt?  
2. Var säljer vi? – Vilka städer står för störst intäkt?  
3. När säljer vi? – Finns säsongsmönster i försäljningen?  
4. Hur ser en typisk order ut? – AOV (Average Order Value).  
5. Topp-3 kategorier efter intäkt.  
6. Eventuella avvikelser.  

In [ ]:
# Importera funktioner 
from src.metrics import total_revenue, total_units, aov, revenue_by_category, revenue_by_city, top_categories
import pandas as pd

In [ ]:
# Visa metrics
print("Total Revenue:", total_revenue(df))
print("Total Units Sold:", total_units(df))
print("Average Order Value (AOV):", aov(df))

print("\nRevenue by Category:\n", revenue_by_category(df))
print("\nRevenue by City:\n", revenue_by_city(df))
print("\nTop Categories:\n", top_categories(df))

In [ ]:
# Beskriva ovan...

In [ ]:
# Visualizations
from src.viz import plot_revenue_by_category, plot_sales_over_time

fig1, ax1 = plot_revenue_by_category(df)
fig2, ax2 = plot_sales_over_time(df)

In [ ]:
# beskriva ovan..

---